# 随机数种子
随机种子（Random Seed）是用于初始化随机数生成器的一个固定值。通过设置相同的随机种子，随机数生成器会产生相同的随机数序列，这对于实验和结果的可重复性至关重要。
常见的随机操作包括：
+ 数据集的随机拆分(如训练集和测试集的划分)
+ 模型参数的随机初始化
+ 数据增强中的随机变换

In [ ]:
# example of setting random seed for reproducibility
# the results will be the same each time
import numpy as np
import random
import torch
seed = 66
np.random.seed(seed)
random.seed(seed)
# torch.manual_seed(seed)
random_matrix_first = np.random.randn(3, 3)
print(random_matrix_first)
random_data_one_first = random.random()
random_data_two_first = random.random()
print(random_data_one_first, random_data_two_first)


In [ ]:
# example of not setting random seed
# the results will be different each time
import numpy as np
import random

random_matrix_second = np.random.randn(3, 3)
print(random_matrix_second)
random_data_one_second = random.random()
random_data_two_second = random.random()
print(random_data_one_second, random_data_two_second)

# argparser
是python标准库中的一个模块，用于解析命令行参数。argparser模块使得编写用户友好的命令行界面变得容易。它可以解析命令行参数并生成帮助信息。argparser模块还可以自动生成帮助信息。

In [ ]:
# example
import argparse
# create the object of ArgumentParser
parser = argparse.ArgumentParser(description = "Description of the program")
# add_argument() method is used to add the arguments
parser.add_argument("--arg1", help = "Description of the arg1")
parser.add_argument("--optional_arg", default = "default_value", help = "Description of the optional_arg")
# parse_args() method is used to parse the arguments
args = parser.parse_args()
# use the arguments
print(args.arg1)
print(args.optional_arg)

# About function "DataLoader" in torch.utils.data

## The parameter "collate_fn" of function "DataLoader" in torch.utils.data
collate_fn：将一个batch的数据样本组织成一个batch，这个batch中的每个数据样本的形状可能不一样，所以需要用一个函数来组织这个batch。默认的collate_fn函数会将数据样本组织成一个list，这样就不会出现形状不一样的问题。如果数据样本的形状一样，可以不用设置collate_fn参数。

作用：
+ 自定义合并逻辑：当数据集中的样本结构复杂或者每个样本的形状不一致时，通过"collate_fn"自定义合并逻辑能够更灵活地处理这些样本。
+ 处理变长序列：对于自然语言处理等任务，样本（例如句子）长度不一，需要自定义合并逻辑进行填充和打包。
+ 处理多模态数据：当一个样本包含多种不同模态的数据（如图像和文本）时，需要自定义合并逻辑以正确处理和打包这些数据。

使用方法：
+ 定义一个函数，该函数接受一个batch的数据样本列表，返回一个batch的数据样本。'collate_fn'函数中的'batch'是一个列表，包含了多个通过'\_\_getitem_\_'方法获取的数据样本。
+ 将该函数传递给DataLoader的collate_fn参数。

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset

class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]

data = [
    {'input': torch.tensor([1, 2, 3]), 'label': torch.tensor(0)},
    {'input': torch.tensor([4, 5]), 'label': torch.tensor(1)},
    {'input': torch.tensor([6, 7, 8, 9]), 'label': torch.tensor(2)}
]

dataset = CustomDataset(data)

def custom_collate_fn(batch):
    inputs = [item['input'] for item in batch]
    labels = [item['label'] for item in batch]

    lengths = [len(item) for item in inputs]
    max_length = max(lengths)

    padded_inputs = torch.zeros((len(inputs), max_length))
    for i, input in enumerate(inputs):
        padded_inputs[i, :len(input)] = input
    
    labels = torch.stack(labels)

    return padded_inputs, labels, lengths
"""
first batch:
[
    {'input': torch.tensor([1, 2, 3]), 'label': torch.tensor(0)},
    {'input': torch.tensor([4, 5]), 'label': torch.tensor(1)}
]

second batch:
[
    {'input': torch.tensor([6, 7, 8, 9]), 'label': torch.tensor(2)}   
]
"""
dataloader = DataLoader(dataset, batch_size = 2, shuffle = False, collate_fn = custom_collate_fn)

for batch in dataloader:
    padded_inputs, labels, lengths = batch
    print("Padded inputs:\n", padded_inputs)
    print("Labels:\n", labels)
    print("Lengths:\n", lengths)
    print("\n")

## The parameter "sampler" and "batch_sampler" of function "DataLoader" in torch.utils.data
sampler: 

定义如何从数据集中采样单个样本。它接受一个"sampler"对象。pytorch中提供了一些内置的采样器:torch.utils.data.RandomSampler, torch.utils.data.SequentialSampler, torch.utils.data.SubsetRandomSampler, torch.utils.data.WeightedRandomSampler等。这些采样器，都接收一个dataset对象，返回一个索引序列，用于获取数据集中的样本。

batch_sampler: 

将单个样本采样器生成的样本索引打包成批次。它接受一个"batch_sampler"对象。pytorch中提供了一些内置的batch采样器:torch.utils.data.BatchSampler, torch.utils.data.WeightedRandomSampler等。这些batch采样器，接收一个sampler对象，返回一个索引序列，用于获取数据集中的样本。

通过这两个参数，pytorch的'DataLoader'提供了灵活且强大的数据加载和采样机制，满足不同的实验和训练要求。另外可以，**自定义'sampler'和'batch_sampler'可以满足特定的数据采样需求**。

自定义'sampler'：
+ 继承torch.utils.data.Sampler类，实现\_\_iter\_\_和\_\_len\_\_方法。

自定义'batch_sampler'：
+ 继承torch.utils.data.BatchSampler类，实现\_\_iter\_\_和\_\_len\_\_方法。

In [ ]:
# example: sampler
import torch
from torch.utils.data import DataLoader, TensorDataset, Dataset

data = torch.arange(10).view(-1, 1)
dataset = TensorDataset(data)

print("Example of sampler")
random_sampler = torch.utils.data.RandomSampler(dataset)
dataloader_one = DataLoader(dataset, sampler = random_sampler, batch_size = 2)
for batch in dataloader_one:
    print(batch)

print('-------------------')

# example: batch_sampler
print("Example of batch sampler")
batch_sampler = torch.utils.data.BatchSampler(random_sampler, batch_size = 3, drop_last = False)
dataloader_two = DataLoader(dataset, batch_sampler = batch_sampler)
for batch in dataloader_two:
    print(batch)

print('-------------------')

# example: customize sampler
class ReverseSampler(torch.utils.data.Sampler):
    def __init__(self, data_source):
        self.data_source = data_source
    
    def __iter__(self):
        return iter(range(len(self.data_source) - 1, -1, -1))
    
    def __len__(self):
        return len(self.data_source)

print("Exampler of customized sampler")
reverse_sampler = ReverseSampler(dataset)
dataloader_three = DataLoader(dataset, sampler = reverse_sampler, batch_size = 2) 

for batch in dataloader_three:
    print(batch)

print('-------------------')

# example: customize batch sampler
class CustomBatchSampler(torch.utils.data.BatchSampler):
    def __init__(self, sampler, batch_size, drop_last):
        self.sampler = sampler
        self.batch_size = batch_size
        self.drop_last = drop_last
    
    def __iter__(self):
        batch = []
        for index in self.sampler:
            batch.append(index)
            if len(batch) == self.batch_size:
                yield batch
                batch = []
        
        if len(batch) > 0 and not self.drop_last:
            yield batch
    
    def __len__(self):
        if self.drop_last:
            return len(self.sampler) // self.batch_size
        else: 
            return (len(self.sampler) + self.batch_size - 1) // self.batch_size

batch_sampler = CustomBatchSampler(reverse_sampler, batch_size = 3, drop_last = False)
dataloader_four = DataLoader(dataset, batch_sampler = batch_sampler)

for batch in dataloader_four:
    print(batch)

# Xavier 初始化
Xavier初始化是用于初始化神经网络权重的一种方法，其目的是让信号能够更好地在网络层间传播，从而避免梯度消失或者爆炸的问题。

Xavier初始化分为均匀分布(Xavier Uniform)和正态分布(Xavier Normal)两种方式。

**Xavier Uniform**:

W ~ U($-\sqrt{\frac{6}{n_{in}+n_{out}}}, \sqrt{\frac{6}{n_{in}+n_{out}}}$)
+ U(a, b)表示均匀分布，a和b分别是分布的上下界。
+ $n_{in}$是输入单元的数量，$n_{out}$是输出单元的数量。

**Xavier Normal**:

W ~ N(0, $\sqrt{\frac{2}{n_{in}+n_{out}}}$)
+ N($\mu$, $\sigma^2$)表示正态分布，$\mu$和$\sigma^2$分别是分布的均值和方差。
+ $n_{in}$是输入单元的数量，$n_{out}$是输出单元的数量。

总结：
+ Xavier Uniform：权重从均匀分布中采样，适用于浅层网络，能够提供平滑的信号传播
+ Xavier Normal： 权重从正态分布中采样，适用于深层网络，能防止梯度消失或爆炸

In [48]:
import torch
import torch.nn as nn
seed = 42
torch.manual_seed(seed)
# Example: Xavier Uniform Initialization
linear = nn.Linear(3, 2)
nn.init.xavier_uniform_(linear.weight).requires_grad_(False)
print(linear.weight)

# Example: Xavier Normal Initialization
linear = nn.Linear(3, 2)
nn.init.xavier_normal_(linear.weight).requires_grad_(False)
print(linear.weight)


Parameter containing:
tensor([[ 0.9657, -0.8036,  0.9522],
        [ 0.2050,  0.8093,  0.1484]])
Parameter containing:
tensor([[-0.8619,  0.8579,  0.4230],
        [-0.4476, -0.2066, -0.1763]])
